## Install Libraries

We will install the software libraries necessary for this week's tutorials, and test them to make sure they work.

If you have never used a command-line or terminal before, [the linked wiki page will help you](https://github.com/DavidGoedicke/RealtimeAudioClassification/wiki/Command-lines-and-terminals-a-quick-introduction.md)

Install [NumPy](https://en.wikipedia.org/wiki/NumPy) (Python support for large, multi-dimensional arrays and matrices ): ```conda install -c conda-forge numpy ```  
Install [MatplotLib](https://matplotlib.org) (Python 2D plotting library ): ```conda install -c conda-forge matplotlib ```  
Install [NumPy Ringbuffer](https://pypi.org/project/numpy_ringbuffer/) (Ringbuffer Data structure library ) ```pip install numpy_ringbuffer```  
Install [pyTorch](https://pytorch.org/get-started/locally/) (Python Machine Learning Library ): ```conda install pytorch torchvision -c pytorch```  
Install [opencv2](https://opencv.org) (Computer Vision library) : ```conda install -c conda-forge  opencv=3.4.1```  
Install [sklearn](https://scikit-learn.org/stable/) (SciKit Machine Learning library): ```conda install scikit-learn```  
Install [librosa](https://librosa.github.io/librosa/) (Music and Audio analysis library ): ```conda install -c conda-forge librosa ```  
Install [PyAudio](https://people.csail.mit.edu/hubert/pyaudio/) (Python bindings for PortAudio, the cross-platform audio I/O library ) : ```conda install -c anaconda pyaudio```  


## Test setup

In the code below, we load ("import") the libraries that we just installed above, to make sure that Python can find the libraries. 


In [ ]:
import csv
import os
import wave
import time
import multiprocessing
from threading import Thread
import os
print("Found all built-in libraries.")
import numpy as np
from numpy_ringbuffer import RingBuffer
import matplotlib.pyplot as plt
print("Found all math libraries.")
import pyaudio
import librosa
print("Found all audio libraries.")
import torch
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
print("Found all machine learning libraries.")
import cv2
print("Found the computer vision library.")


If the above code runs without a problem, all libraries have installed and can be loaded. While it does not gurantee that everything will work, this is a very good sign.
